在开始介绍 pytorch 的一个使用语法结构前,我们先说一下 深度学习领域几位真正的大神,可以说深度学习领域的目前的产出都离不开这几位大神
Hinton, LeCun, Bengio

Hinton 杰弗里·埃弗里斯特·辛顿
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/34/Geoffrey_Hinton_at_UBC.jpg/440px-Geoffrey_Hinton_at_UBC.jpg" style="zoom:50%;margin-left:0px" />

杰弗里·埃弗里斯特·辛顿 FRS（英语：Geoffrey Everest Hinton）（1947年12月6日－）是一位英国出生的加拿大计算机学家和心理学家，以其在神经网络方面的贡献闻名。辛顿是反向传播算法和对比散度算法的发明人之一，也是深度学习的积极推动者

后面两位就不细说了,总之都是很牛,可以说这三位以及他们的学生 点亮了深度学习领域的一盏明灯







### pytorch 基础及重要 api 介绍

首先 推荐优先使用虚拟环境,接下来我们准备安装依赖包 

```
pip3 install http://download.pytorch.org/whl/torch-0.3.1-cp36-cp36m-macosx_10_7_x86_64.whl 
pip3 install torchvision 
```

1.1.1 Tensor 张量
Pytorch 里面的基本对象就是张量 Tensor, 其表示的是已给多维的矩阵,其和 numpy 的数组是对应的 Tensor ,二者的区别就是 Pytorch 的 Tensor 可以运行在 GPU 上, TensorFlow 的 tensor 也是

依据数据类型 tensor 分一下几种
- torch.FloatTensor 32
- torch.DoubleTensor 64
- torch.shortTensor 16
- torch.IntTensor 32

下面我们给一些实例说明演示一下

In [1]:
import torch
a = torch.FloatTensor([[2,3], [4,8], [7,9]])
print("{}".format(a))
print("a size is {}".format(a.size()))


 2  3
 4  8
 7  9
[torch.FloatTensor of size 3x2]

a size is torch.Size([3, 2])
